# Running the Code Samples in the Course

Thoughout the course, we will be using
LiteLLM
, a library that provides a unified interface for interacting with over 100 different LLMs including GPT-4, Claude, Gemini, and many others. By using a single interface, we can easily switch between different models without changing our code, making it perfect for testing different LLMs or having fallback options in production. This will also allow us to make our agent reusable across LLMs — although we may need to do some prompt engineering to make it work with a new LLM.



In [3]:
import os
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = api_key

# Programmatic Prompting for Agents

## Sending Prompts Programmatically & Managing Memory
**Note: a link to the complete code in a Google Colab notebook will be provided in the next item in the course. Read through this lesson and then click next to get the link to the code.**

To get started building agents, we need to understand how to send prompts to LLMs. Agents require two key capabilities:

1. **Programmatic prompting** - Automating the prompt-response cycle that humans do manually in a conversation. This forms the foundation of the Agent Loop we’ll explore.

2. **Memory managemen**t - Controlling what information persists between iterations, like API calls and their results, to maintain context through the agent’s decision-making process.

Programmatically sending prompts is how we move from having a human type in prompts and then take action based on the LLM’s response to having an agent that can do this automatically. The Agent Loop that we will begin building over the next several readings will be programmatically sending prompts to the LLM and then taking action based on the LLM’s response.

We will also need to understand how to manage what the LLM knows or remembers. This is important because we want to be able to control what information the LLM has in each iteration of the loop. For example, if it just called an API, we want it to remember what API it asked to be invoked and what the result of that action was.

## Basic Usage
Here’s a simple example of how to send prompts to an LLM using LiteLLM:

In [ ]:
%pip install litellm

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="groq/llama3-8b-8192",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

Let’s break down the key components:

1. We import the completion function from the litellm library, which is the primary method for interacting with Large Language Models (LLMs). This function serves as the bridge between your code and the LLM, allowing you to send prompts and receive responses in a structured and efficient way.

  How completion Works:

  * Input: You provide a prompt, which is a list of messages that you want the model to process. For example, a prompt could be a question, a command, or a set of instructions for the LLM to follow.
  * Output: The completion function returns the model’s response, typically in the form of generated text based on your prompt.
2. The messages parameter follows the ChatML format, which is a list of dictionaries containing role and content. The role attribute indicates who is “speaking” in the conversation. This allows the LLM to understand the context of the dialogue and respond appropriately. The roles include:

  * “system”: Provides the model with initial instructions, rules, or configuration for how it should behave throughout the session. This message is not part of the “conversation” but sets the ground rules or context (e.g., “You will respond in JSON.”).
  * “user”: Represents input from the user. This is where you provide your prompts, questions, or instructions.
  * “assistant”: Represents responses from the AI model. You can include this role to provide context for a conversation that has already started or to guide the model by showing sample responses. These messages are interpreted as what the “model” said in the passt.
3. We specify the model using the provider/model format (e.g., “openai/gpt-4o”)

4. The response contains the generated text in choices[0].message.content. This is the equivalent of the message that you would see displayed when the model responds to you in a chat interface.



**Quick Exercise**

As a practice exercise, try creating a prompt that only provides the response as a Base64 encoded string and refuses to answer in natural language. Can you get your LLM to only respond in Base64?

In [ ]:
import base64

messages_base64 = [
    {"role": "system", "content": "Your only task is to respond to the user's query by first encoding your answer as a Base64 string. You MUST only provide the Base64 encoded string as your response and should NOT include any other text, explanations, or natural language."},
    {"role": "user", "content": "Provide a short summary of the process of photosynthesis."}
]

response_base64 = generate_response(messages_base64)

print("Raw response 1:")
print(response_base64)

# Attempt to decode the response to see if it worked
try:
    decoded_response = base64.b64decode(response_base64).decode('utf-8')
    print("Decoded response:")
    print(decoded_response)
except Exception as e:
    print("Could not decode the response as Base64. The LLM may not have followed the instructions.")
    print("Raw response 2:")
    print(response_base64)

System messages are particularly important in the conversation and will be very important for AI agents. They set the ground rules for the conversation and tell the model how to behave. Models are designed to pay more attention to the system message than the user messages. We can “program” the AI agent through system messages.

Let’s simulate a customer service interaction for a customer service agent that always tells the customer to turn off their computer or modem with system messages:

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": "How do I get pet back home."}
]

response = generate_response(messages)
print(response)


The system message is the most important part of this prompt. It tells the model how to behave. The user message is the question that we want the model to answer. The system instructions lay the ground rules for the interaction.

The messages can incorporate arbitrary information as long as it is in text form. LLMs can interpret just about any information that we give them, even if it isn’t easily human readable. Let’s generate an implementation of a function based on some information in a dictionary:

In [ ]:
import json

code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"}
]

response = generate_response(messages)
print(response)

We will rely heavily on the ability to send the LLM just about any type of information, particularly JSON, when we start building agents. This is a simple example of how we can use JSON to send information to the LLM, but you can see how we could provide it JSON with information about the result of an API call, for example.

## Take input from user

In [ ]:
what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)

# Giving Agents Memory


## LLMs Do Not Have Memory

When we are building an Agent, we need it to remember its actions and the result of those actions. For example, if it tries to create a calendar event for a meeting and the API call fails due to an incorrect parameter value that it provided, we want it to remember that the API call failed and why. This way, it can correct the mistake and try again. If we have a complex task that we break down into multiple steps, we need the Agent to remember the results of each step to ensure that it can continue the task from where it left off. Memory is crucial for Agents.

### LLMs Do Not Have Memory

When interacting with an LLM, the model does not inherently “remember” previous conversations or responses. Every time you call the model, it generates a response based solely on the information provided in the messages parameter. If previous context is not included in the messages, the model will not have any knowledge of it.

This means that to simulate continuity in a conversation, you must explicitly pass all relevant prior messages (including system, user, and assistant roles) in the messages list for each request.

**Example 1: Missing Context in the Prompt**



In [ ]:
messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

# Second query without including the previous response
messages = [
    {"role": "user", "content": "Update the function to include documentation."}
]

response = generate_response(messages)
print(response)

**Explanation:** In the second request, the model doesn’t “remember” the function it wrote in the first interaction. Since the information is not included in the second prompt, the model cannot connect the two.

**Example 2: Including Previous Responses for Continuity**

To fix this issue, we need to add new messages with the “assistant” role to the messages list with the content of the prior response from the LLM. This way, the model can see what code it wrote previously and can build on that.

In [ ]:
messages = [
   {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
   {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

# We are going to make this verbose so it is clear what
# is going on. In a real application, you would likely
# just append to the messages list.
messages = [
   {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
   {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."},

   # Here is the assistant's response from the previous step
   # with the code. This gives it "memory" of the previous
   # interaction.
   {"role": "assistant", "content": response},

   # Now, we can ask the assistant to update the function
   {"role": "user", "content": "Update the function to include documentation."}
]

response = generate_response(messages)
print(response)

**Explanation:** By including the assistant’s previous response in the messages, the model can maintain context and provide an appropriate response to the follow-up question.

**Key Takeaways**

1. **No Inherent Memory:** The LLM has no knowledge of past interactions unless explicitly provided in the current prompt (via messages).
2. **Provide Full Context:** To simulate continuity in a conversation, include all relevant messages (both user and assistant responses) in the messages parameter.
3. **Role of Assistant Messages:** Adding previous responses as assistant messages allows the model to maintain a coherent conversation and build on earlier exchanges. For an agent, this will allow it to remember what actions, such as API calls, it took in the past.
4. **Memory Management:** We can control what the LLM remembers or does not remember by managing what messages go into the conversation. Causing the LLM to forget things can be a powerful tool in some circumstances, such as when we need to break a pattern of poor responses from an Agent.

**Why This Matters**

Understanding the stateless nature of LLMs is crucial for designing agents that rely on multi-turn conversations with their environment. Developers must explicitly manage and provide context to ensure the model generates accurate and relevant responses.

# Practicing Programmatic Prompting for Agents

## Building a Quasi-Agent

For practice, we are going to write a quasi-agent that can write Python functions based on user requirements. It isn’t quite a real agent, it can’t react and adapt, but it can do something useful for us.

The quasi-agent will ask the user what they want code for, write the code for the function, add documentation, and finally include test cases using the unittest framework. This exercise will help you understand how to maintain context across multiple prompts and manage the information flow between the user and the LLM. It will also help you understand the pain of trying to parse and handle the output of an LLM that is not always consistent.



## Practice Exercise

This exercise will allow you to practice programmatically sending prompts to an LLM and managing memory.

For this exercise, you should write a program that uses sequential prompts to generate any Python function based on user input. The program should:

1. First Prompt:

  * Ask the user what function they want to create
  * Ask the LLM to write a basic Python function based on the user’s description
  * Store the response for use in subsequent prompts
  * Parse the response to separate the code from the commentary by the LLM

2. Second Prompt:

  * Pass the code generated from the first prompt
  * Ask the LLM to add comprehensive documentation including:
    * Function description
    * Parameter descriptions
    * Return value description
    * Example usage
    * Edge cases
3. Third Prompt:

  * Pass the documented code generated from the second prompt
  * Ask the LLM to add test cases using Python’s unittest framework
  * Tests should cover:
    * Basic functionality
    * Edge cases
    * Error cases
    * Various input scenarios
Requirements:

* Use the LiteLLM library
* Maintain conversation context between prompts
* Print each step of the development process
* Save the final version to a Python file

If you want to practice further, try using the system message to force the LLM to always output code that has a specific style or uses particular libraries.

### Task Solution
Create a Python function based on a user-provided description, add documentation and unit tests, and save the combined code to a file.

#### Get user input for the function description

##### Subtask:
Prompt the user to describe the Python function they want to create.


**Reasoning**:
The subtask is to prompt the user for a function description and store it in a variable. The provided code block already does this using the `input()` function.



In [37]:
import re

def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

The LLM often includes commentary with its code. This function extracts just the code block, making it easier to build upon in subsequent prompts.

In [ ]:
function_description = input("What function do you want to create?")

messages = [
    {"role": "system", "content": "You are a Python expert helping to develop a function."}
]

messages.append({
    "role": "user",
    "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
})

#### Generate initial function code

##### Subtask:
Use LiteLLM with a system message to generate the initial Python function code based on the user's description. Extract the code block from the LLM's response.


**Reasoning**:
Call the generate_response function with the messages list to get the initial code, then extract and print the code block.



In [ ]:
initial_response = generate_response(messages)
print("Initial response:")
print(initial_response)

initial_code = extract_code_block(initial_response)
print("\nExtracted initial code:")
print(initial_code)

#### Generate documented code

##### Subtask:
Use LiteLLM with an updated system message and the previously generated code to add comprehensive documentation to the function. Extract the documented code block.


**Reasoning**:
Define a new list of messages for the second prompt, including the previous conversation context and the instruction to add comprehensive documentation. Then, call the generate_response function to get the documented code and extract the code block using the extract_code_block function.



In [ ]:
messages_documented = [
   {"role": "system", "content": "You are a Python expert helping to develop a function."},
   {"role": "user", "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."},
   {"role": "assistant", "content": initial_response},
   {"role": "user", "content": """Update the following Python function to include comprehensive documentation. The documentation should include:
- Function description
- Parameter descriptions
- Return value description
- Example usage
- Edge cases
Output the documented function in a ```python code block```."""}
]

documented_response = generate_response(messages_documented)
print("Documented response:")
print(documented_response)

documented_code = extract_code_block(documented_response)
print("\nExtracted documented code:")
print(documented_code)


#### Generate test cases

##### Subtask:
Use LiteLLM with another updated system message and the documented code to generate unit tests for the function using the `unittest` framework. Extract the test code block.


**Reasoning**:
Use LiteLLM with another updated system message and the documented code to generate unit tests for the function using the `unittest` framework, and extract the test code block.



In [ ]:
messages_tested = [
   {"role": "system", "content": "You are a Python expert helping to develop a function."},
   {"role": "user", "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."},
   {"role": "assistant", "content": initial_response},
   {"role": "user", "content": """Update the following Python function to include comprehensive documentation. The documentation should include:
- Function description
- Parameter descriptions
- Return value description
- Example usage
- Edge cases
Output the documented function in a ```python code block```."""},
    {"role": "assistant", "content": documented_response},
    {"role": "user", "content": """Write unit tests for the following Python function using the `unittest` framework. The tests should cover:
- Basic functionality
- Edge cases
- Error cases
- Various input scenarios
Output the test code in a ```python code block```."""}
]

tested_response = generate_response(messages_tested)
print("Tested response:")
print(tested_response)

test_code = extract_code_block(tested_response)
print("\nExtracted test code:")
print(test_code)


#### Combine code and tests

##### Subtask:
Combine the documented function code and the generated test cases into a single string.


**Reasoning**:
Combine the documented function code and the generated test cases into a single string as per the instructions.



In [ ]:
combined_code = documented_code + "\n\n" + test_code
print("Combined code:")
print(combined_code)

#### Save to file

##### Subtask:
Save the combined code to a Python file.


**Reasoning**:
Save the combined code to a Python file.



In [43]:
file_name = "my_function_with_tests.py"
with open(file_name, 'w') as f:
    f.write(combined_code)

#### Summary:

##### Data Analysis Key Findings

*   The user's request for a Python function was successfully captured using the `input()` function.
*   LiteLLM was effectively used in multiple steps to generate the initial function code, add comprehensive documentation (including description, parameters, return value, examples, and edge cases), and create unit tests using the `unittest` framework (covering basic functionality, edge cases, error cases, and various input scenarios).
*   A custom `extract_code_block` function successfully extracted the Python code from the LLM responses at each stage.
*   The documented function code and the generated test code were successfully combined into a single string.
*   The combined code was successfully saved to a Python file named "my\_function\_with\_tests.py".

##### Insights or Next Steps

*   The process demonstrates a robust workflow for using an LLM to generate, document, and test Python functions based on a user description.
*   The generated Python file ("my\_function\_with\_tests.py") is now ready to be executed to verify the function's behavior and the correctness of the tests.
